# 0 Configuration

In [ ]:
config = {
    "data_loading": {
        "merge_prep": {
            "diff_plot": False
        }
    },
    "overview": {
        "overview_plot": False
    },
    "cleaning": {
        "emptyness": {
            "overview_plot": False
        },
        "usability": {
            "numericals": {
                "overview_plot": False,
                "outliers": {
                    "remove": True,
                    "plot": False
                },
                "scaling": {
                    "distrib_plot": False
                }
            },
            "categoricals": {
                "inconsistencies": {
                    "to_num": True,
                    "to_bool": True,
                    "strip_and_lower": True
                }
            }
        }
    },
    "correlation": {
        "heatmap_pre_plot": False,
        "correlation_process": False,
        "heatmap_post_plot": False
    }
}

***
# 1 Dependency import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from fuzzywuzzy import process
import chardet

from math import ceil

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

np.random.seed(0)

In [ ]:
def dataframe_distribution_overview(data, figsize=(10, 3)):
    plt.figure(figsize=figsize)

    sns.barplot(x=data.columns, y=data.count())

    plt.title("Number of values per column", size=20)
    plt.xticks(rotation=45, size=16, ha="right")
    plt.yticks(size=16)
    plt.ylabel("Number values", size=16)
    plt.show()

In [ ]:
def index_lth(data, percentage: int):
    percentage = percentage / 100
    less_than = data.count() < data.shape[0] * percentage
    index_less_than = less_than[less_than == True].index
    return index_less_than

In [ ]:
def multi_plot_numerical(data, features, kind="hist", n_cols=8, figsize=(30, 10), wspace=0.35, hspace=0.35):
    feature_nb = len(features)
    n_rows = ceil(feature_nb / n_cols)
    index = 0

    plt.subplots(n_rows, n_cols, figsize=figsize)
    plt.subplots_adjust(wspace=wspace, hspace=hspace)

    for r in range(n_rows):
        for c in range(n_cols):
            if index >= feature_nb:
                break

            plt.subplot(n_rows, n_cols, index+1)

            feature = features[index]

            if kind == "box":
                plot = sns.boxplot(y=data[feature])
            elif kind == "hist":
                plot = sns.histplot(data=data[feature], kde=True)
            else:
                plot = sns.histplot(data=data[feature], kde=True)

            plot.set_xlabel(feature, fontsize=12)
            plot.set_ylabel(None)

            index += 1

    plt.show()

In [ ]:
def create_subplot(dataset, feature, n_rows, index, is_numeric):
    plt.subplot(n_rows, 2, index)
    uniques = dataset[feature].unique()

    if is_numeric:
        plot = sns.boxplot(y=dataset[feature])
    else:
        if uniques.size <= 20 and uniques.size > 0:
            plot = sns.countplot(x=dataset[feature])
            plt.xticks(rotation=45, size=8, ha="right")
        else:
            dist = pd.DataFrame(data=[[uniques.size, dataset.shape[0] - uniques.size]], columns=["uniques", "not_uniques"])
            plot = sns.barplot(data=dist)

    plot.set_xlabel(None)
    plot.set_ylabel(feature, fontsize=14)


def dataset_diff_analysis(data1, data2, exclude=[], figsize=(15, 200)):
    features = data1.columns.union(data2.columns).difference(exclude)
    n_cols = 2
    n_rows = len(features)
    col_id = 0
    index = 1

    plt.subplots(n_rows, n_cols, figsize=figsize)
    plt.subplots_adjust(wspace=0.35, hspace=0.5)

    for f in features:
        is_numeric = False
        if f in data1.columns:
            f_type = data1[f].dtype
        else:
            f_type = data2[f].dtype
        if f_type in ["int64", "float64"]:
            is_numeric = True

        if f in data1.columns.values:
            create_subplot(data1, f, n_rows, index, is_numeric)
        if f in data2.columns.values:
            create_subplot(data2, f, n_rows, index+1, is_numeric)

        index += 2

    print(f"features: {features}")
    plt.show()

In [ ]:
def decribe_several(feature, *df):
    data = {}
    index = 1
    for d in df:
        data[feature + "_" + str(index)] = d[feature]
        index += 1
    final_dataframe = pd.DataFrame(data)
    return final_dataframe.describe()

In [ ]:
def head_several(feature, nb, *df):
    data = {}
    index = 1
    for d in df:
        data[feature + "_" + str(index)] = d[feature]
        index += 1
    final_dataframe = pd.DataFrame(data)
    return final_dataframe.head(nb)

In [ ]:
class OutlierProcessor():
    def __init__(self, data, feature, lower_trig, upper_trig):
        self.data = data
        self.feature = feature
        self.lower_trig = lower_trig
        self.upper_trig = upper_trig
        self.__above = 0
        self.__below = 0
        self.__total = 0

    def __print(self):
        print(f"lower_trig: {self.lower_trig}")
        print(f"upper_trig: {self.upper_trig}")
        print(f"below: {self.__below}")
        print(f"above: {self.__above}")
        print(f"total: {self.__total}")

    def count(self):
        self.__below = self.data[self.data[self.feature] < self.lower_trig][self.feature].count()
        self.__above = self.data[self.data[self.feature] > self.upper_trig][self.feature].count()
        self.__total = self.__below + self.__above
        self.__print()

    def replace(self, replace_by=np.nan, inplace=False):
        return self.data[self.feature].where(cond=lambda x: ((x > self.lower_trig) & (self.upper_trig > x)), other=replace_by, inplace=inplace)


In [ ]:
class OutlierIqrProcessor(OutlierProcessor):
    def __init__(self, data, feature):
        self.__q1 = data[feature].quantile(0.25)
        self.__q3 = data[feature].quantile(0.75)
        self.__iqr = self.__q3 - self.__q1
        upper_trig = self.__q3 + (1.5 * self.__iqr)
        lower_trig = self.__q1 - (1.5 * self.__iqr)
        super().__init__(data, feature, lower_trig, upper_trig)


In [ ]:
def correlation_heatmap(dataset, figsize=(30, 20)):
    plt.figure(figsize=figsize)

    correlation = dataset.corr()
    mask = np.triu(np.ones_like(correlation, dtype=bool))

    sns.heatmap(data=correlation, mask=mask, annot=True, vmax=.75, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

    plt.title("Correlation heatmap", size=20)
    plt.xticks(rotation=45, size=16, ha="right")
    plt.yticks(size=16)
    plt.show()

In [ ]:
def unique_several(dataset, features, take=20):
    uniques_data = {}
    too_many_uniques = []
    only_one_uniques = []
    only_two_uniques = []

    for f in features:
        f_uniques = dataset[f].unique()
        if f_uniques.size <= take:
            if f_uniques.size == 1:
                only_one_uniques.append(f)
            elif f_uniques.size == 2:
                only_two_uniques.append(f)
            else:
                uniques_data[f] = pd.Series(data=f_uniques, name=f, dtype="object")
        else:
            too_many_uniques.append((f, f_uniques.size))

    print(f"Only one unique: {only_one_uniques}")
    print(f"Only two uniques: {only_two_uniques}")
    print(f"Too many uniques: {too_many_uniques}")

    return pd.DataFrame(data=uniques_data)

In [ ]:
def strip_and_lower(dataset, features, to_strip=None, inplace=False):
    result_data = {}
    for f in features:
        result_data[f] = pd.Series(data=dataset[f].str.strip(to_strip=to_strip), name=f, dtype="object")
        result_data[f] = result_data[f].str.lower()
    result = pd.DataFrame(data=result_data)
    if inplace:
        dataset[features] = result
    else:
        return result

In [ ]:
def fuzzy_matching_several(dataset, fuzzy, limit=10):
    fuzzy_data = {}
    for feature, match in fuzzy:
        fuzzy_matches = fuzzywuzzy.process.extract(match, dataset[feature], limit=limit, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
        fuzzy_data[feature] = pd.Series(data=fuzzy_matches, name=feature, dtype="object")
    return pd.DataFrame(data=fuzzy_data)

***
# 2 Data loading

## 2.1 Loading

In [ ]:
data = pd.read_csv("data/2015-building-energy-benchmarking.csv", delimiter=",")

In [ ]:
data2 = pd.read_csv("data/2016-building-energy-benchmarking.csv", delimiter=",")

***
## 2.2 Merge preparation

In [ ]:
if config["data_loading"]["merge_prep"]["diff_plot"]:
    dataset_diff_analysis(data, data2, exclude=["Comment", "Comments"], figsize=(15, 200))

***
### 2.2.1 '2010 Census Tracts'

In [ ]:
feature = "2010 Census Tracts"

In [ ]:
data[feature].describe()

***
### 2.2.2 Address

In [ ]:
feature = "Address"

In [ ]:
data2[feature].describe()

***
### 2.2.3 BuildingType

In [ ]:
feature = "BuildingType"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.4 City

In [ ]:
feature = "City"

In [ ]:
data2[feature].describe()

***
### 2.2.5 'City Council Districts'

In [ ]:
feature = "City Council Districts"

In [ ]:
data[feature].describe()

***
### 2.2.6 ComplicanceStatus

In [ ]:
feature = "ComplianceStatus"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.7 CouncilDistrictCode

In [ ]:
feature = "CouncilDistrictCode"

In [ ]:
decribe_several(feature, data, data2)

In [ ]:
data.drop(columns=[feature], inplace=True)

***
### 2.2.8 DataYear

In [ ]:
feature = "DataYear"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.9 DefaultData

In [ ]:
feature = "DefaultData"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.10 ENERGYSTARScore

In [ ]:
feature = "ENERGYSTARScore"

In [ ]:
decribe_several(feature, data, data2)

In [ ]:
data.drop(columns=[feature], inplace=True)

***
### 2.2.11 Electricity(kBtu)

In [ ]:
feature = "Electricity(kBtu)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.12 Electricity(kWh)

In [ ]:
feature = "Electricity(kWh)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.13 GHGEmissions(MetricTonsCO2e)

In [ ]:
feature = "GHGEmissions(MetricTonsCO2e)"

In [ ]:
data[feature].describe()

***
### 2.2.14 GHGEmissionsIntensity

In [ ]:
feature = "GHGEmissionsIntensity"

In [ ]:
data2[feature].describe()

***
### 2.2.15 GHGEmissionsIntensity(KgCO2e/ft2)

In [ ]:
feature = "GHGEmissionsIntensity(kgCO2e/ft2)"

In [ ]:
data[feature].describe()

***
### 2.2.16 LargestPropertyUseType

In [ ]:
feature = "LargestPropertyUseType"

In [ ]:
decribe_several(feature, data, data2)

In [ ]:
data.drop(columns=[feature], inplace=True)

***
### 2.2.17 LargestPropertyUseTypeGFA

In [ ]:
feature = "LargestPropertyUseTypeGFA"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.18 Latitude

In [ ]:
feature = "Latitude"

In [ ]:
data2[feature].describe()

***
### 2.2.19 ListOfAllPropertyUseTypes

In [ ]:
feature = "ListOfAllPropertyUseTypes"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.20 Location

In [ ]:
feature = "Location"

In [ ]:
data[feature].describe()

In [ ]:
data.drop(columns=[feature], inplace=True)

***
### 2.2.21 Longitude

In [ ]:
feature = "Longitude"

In [ ]:
data2[feature].describe()

***
### 2.2.22 NaturalGas(kBtu)

In [ ]:
feature = "NaturalGas(kBtu)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.23 NaturalGas(therms)

In [ ]:
feature = "NaturalGas(therms)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.24 Neighborhood

In [ ]:
feature = "Neighborhood"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.25 NumberofBuildings

In [ ]:
feature = "NumberofBuildings"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.26 NumberofFloors

In [ ]:
feature = "NumberofFloors"

In [ ]:
decribe_several(feature, data, data2)

In [ ]:
data.drop(columns=[feature], inplace=True)

***
### 2.2.27 OtherFuelUse(kBtu)

In [ ]:
feature = "OtherFuelUse(kBtu)"

In [ ]:
data[feature].describe()

***
### 2.2.28 Outlier

In [ ]:
feature = "Outlier"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.29 PrimaryPropertyType

In [ ]:
feature = "PrimaryPropertyType"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.30 PropertyGFABuilding(s)

In [ ]:
feature = "PropertyGFABuilding(s)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.31 PropertyGFAParking

In [ ]:
feature = "PropertyGFAParking"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.32 PropertyGFATotal

In [ ]:
feature = "PropertyGFATotal"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.33 PropertyName

In [ ]:
feature = "PropertyName"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.34 SPD Beats

In [ ]:
feature = "SPD Beats"

In [ ]:
data[feature].describe()

***
### 2.2.35 'Seattle Police Department Micro Community Policing Plan Areas'

In [ ]:
feature = "Seattle Police Department Micro Community Policing Plan Areas"

In [ ]:
data[feature].describe()

***
### 2.2.36 SecondLargestPropertyUseType

In [ ]:
feature = "SecondLargestPropertyUseType"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.37 SecondLargestPropertyUseTypeGFA

In [ ]:
feature = "SecondLargestPropertyUseTypeGFA"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.38 SiteEUI(kBtu/sf)

In [ ]:
feature = "SiteEUI(kBtu/sf)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.39 SiteEUIWN(kBtu/sf)

In [ ]:
feature = "SiteEUIWN(kBtu/sf)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.40 SiteEnergyUse(kBtu)

In [ ]:
feature = "SiteEnergyUse(kBtu)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.41 SiteEnergyUseWN(kBtu)

In [ ]:
feature = "SiteEnergyUseWN(kBtu)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.42 SourceEUI(kBtu/sf)

In [ ]:
feature = "SourceEUI(kBtu/sf)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.43 SourceEUIWN(kBtu/sf)

In [ ]:
feature = "SourceEUIWN(kBtu/sf)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.44 State

In [ ]:
feature = "State"

In [ ]:
data2[feature].describe()

***
### 2.2.45 SteamUse(kBtu)

In [ ]:
feature = "SteamUse(kBtu)"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.46 TaxParcelIdentificationNumber

In [ ]:
feature = "TaxParcelIdentificationNumber"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.47 ThirdLargestPropertyUseType

In [ ]:
feature = "ThirdLargestPropertyUseType"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.48 ThirdLargestPropertyUseTypeGFA

In [ ]:
feature = "ThirdLargestPropertyUseTypeGFA"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.49 TotalGHGEmissions

In [ ]:
feature = "TotalGHGEmissions"

In [ ]:
data2[feature].describe()

***
### 2.2.50 YearBuilt

In [ ]:
feature = "YearBuilt"

In [ ]:
decribe_several(feature, data, data2)

In [ ]:
data.drop(columns=[feature], inplace=True)

***
### 2.2.51 YearsENERGYSTARCertified

In [ ]:
feature = "YearsENERGYSTARCertified"

In [ ]:
decribe_several(feature, data, data2)

***
### 2.2.52 'Zip Codes'

In [ ]:
feature = "Zip Codes"

In [ ]:
data[feature].describe()

A research show that the Zip code of Seattle are like: 98***

In [ ]:
data.drop(columns=[feature], inplace=True)

***
### 2.2.53 ZipCode

In [ ]:
feature = "ZipCode"

In [ ]:
data2[feature].describe()

***
### 2.2.54 Comment

In [ ]:
feature = "Comment"

In [ ]:
data[feature].describe()

In [ ]:
data.drop(columns=[feature], inplace=True)

***
### 2.2.55 Comments

In [ ]:
feature = "Comments"

In [ ]:
data2[feature].describe()

In [ ]:
data2.drop(columns=[feature], inplace=True)

***
## 2.4 Merge

In [ ]:
data = data.merge(
    data2, 
    how="outer", 
    on="OSEBuildingID", 
    suffixes=("_2015", "_2016"))

In [ ]:
len(data.columns)

In [ ]:
data.columns.tolist()

***
# 3 Overview

In [ ]:
if config["overview"]["overview_plot"]:
    dataframe_distribution_overview(data, figsize=(30, 3))

***
# 4 Cleaning

## 4.1 Target

In [ ]:
targets = ["SiteEnergyUse(kBtu)_2015", "SiteEnergyUse(kBtu)_2016"]

In [ ]:
targets_na = data[targets].isna()
targets_na.sum()

In [ ]:
nb_rows_before_drop = data.shape[0]

In [ ]:
data.dropna(subset=targets, inplace=True)

In [ ]:
nb_rows_after_drop = data.shape[0]

In [ ]:
targets_na = data[targets].isna()
targets_na.sum()

In [ ]:
print(f"rows dropped: {nb_rows_before_drop - nb_rows_after_drop}")

***
## 4.1 Emptyness

In [ ]:
cols_to_remove = index_lth(data, 10)

In [ ]:
data.drop(columns=cols_to_remove, inplace=True)

In [ ]:
if config["cleaning"]["emptyness"]["overview_plot"]:
    dataframe_distribution_overview(data, figsize=(30, 3))

***
## 4.2 Categoricals

### 4.2.1 Overview

In [ ]:
categorical_data = data.select_dtypes("object")
categorical_data.describe()

In [ ]:
categorical_data.head()

***
### 4.2.2 Types

feature "TaxParcelIdentificationNumber_2015" is supposed to be a numeric type but a value force pandas to make it 'object'.  
It must be parsed and each conversion error will lead to convert the value to NaN

In [ ]:
if config["cleaning"]["usability"]["categoricals"]["inconsistencies"]["to_num"]:
    data["TaxParcelIdentificationNumber_2015"] = pd.to_numeric(data["TaxParcelIdentificationNumber_2015"], errors="coerce")

In [ ]:
data["TaxParcelIdentificationNumber_2015"].describe()

feature "DefaultData_2015" can be seen as a 'bool' type.  
First: replace all the 'No' and 'Yes' by real boolean values  
Second: make sure the feature is now 'bool' type

In [ ]:
data["DefaultData_2015"].unique()

In [ ]:
if config["cleaning"]["usability"]["categoricals"]["inconsistencies"]["to_bool"]:
    data.replace({"DefaultData_2015": {"No": False, "Yes": True}}, inplace=True)

In [ ]:
data["DefaultData_2015"].unique()

In [ ]:
data["DefaultData_2015"].dtype

feature "DefaultData_2016" has boolean values but is of type "object", it must be convert.

In [ ]:
if config["cleaning"]["usability"]["categoricals"]["inconsistencies"]["to_bool"]:
    data["DefaultData_2016"] = data["DefaultData_2016"].astype("bool")

In [ ]:
data["DefaultData_2016"].dtype

Now some features has been converted to other type than 'object', we must reevaluate or categorical_data

In [ ]:
categorical_data = data.select_dtypes("object")

***
### 4.2.3 Inconsistencies

In [ ]:
unique_several(data, categorical_data.columns.values, take=50)

In [ ]:
if config["cleaning"]["usability"]["categoricals"]["inconsistencies"]["strip_and_lower"]:
    strip_and_lower(data, categorical_data.columns.values, inplace=True)

In [ ]:
unique_several(data, categorical_data.columns.values, take=50)

In [ ]:
# fuzzy = [
#     ("Neighborhood_2016", "Ballard"),
# ]
# fuzzy_matching_several(data)

***
## 4.3 Numericals

### 4.3.1 Overview

In [ ]:
numerical_data = data.select_dtypes(["int64", "float64"])

In [ ]:
if config["cleaning"]["usability"]["numericals"]["overview_plot"]:
    multi_plot_numerical(data, numerical_data.columns.values, kind="box", n_cols=6, hspace=0.4, wspace=0.30, figsize=(30, 25))

***
### 4.3.2 Outliers removal

In [ ]:
values = data[numerical_data.columns.values].idxmax().value_counts()
# values

In [ ]:
data[numerical_data.columns.values].describe()

In [ ]:
data.loc[values[values > 3].index]

In [ ]:
if config["cleaning"]["usability"]["numericals"]["outliers"]["remove"]:
    data.drop(index=values[values > 3].index, inplace=True)

In [ ]:
if config["cleaning"]["usability"]["numericals"]["outliers"]["plot"]:
    multi_plot_numerical(data, numerical_data.columns.values, kind="box", n_cols=6, hspace=0.4, wspace=0.30, figsize=(30, 25))

***
### 4.3.3 Scaling

In [ ]:
if config["cleaning"]["usability"]["numericals"]["scaling"]["distrib_plot"]:
    multi_plot_numerical(data, numerical_data.columns.values, n_cols=5, hspace=0.4, wspace=0.2, figsize=(30, 70))

***
# 5 Correlations

In [ ]:
if config["correlation"]["heatmap_pre_plot"]:
    correlation_heatmap(data)

In [ ]:
def categorize(feature):
    new_feature = feature.astype("category").cat.codes
    return new_feature

if config["correlation"]["correlation_process"]:
    categs = data.select_dtypes(include="object")
    categs_to_nums = categs.apply(lambda col: categorize(col))

    for col in categs_to_nums.columns:
        categs_to_nums.rename(columns={col: col + "_CATEG"}, inplace=True)

    data_enhanced = data.join([categs_to_nums])

In [ ]:
if config["correlation"]["heatmap_post_plot"]:
    correlation_heatmap(data_enhanced)

***
# 6 Saving

In [ ]:
data.to_csv("data/data-cleaned.csv", sep=",")

***
# 7 ...